Step 1: Resize all images at once so PIL doesn't get overwhelmed later

In [8]:
import os
from PIL import Image

def resize_images_in_folder(folder_path, size=(512, 512)):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
                img_path = os.path.join(root, file)
                try:
                    with Image.open(img_path) as img:
                        img = img.resize(size, Image.LANCZOS)
                        img.save(img_path)
                        print(f"Resized {img_path}")
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

if __name__ == "__main__":
    base_folders = [
        "../DATASETS/amalgam/Testing/fire",
        "../DATASETS/amalgam/Testing/nofire",
        "../DATASETS/amalgam/Training and Validation/fire",
        "../DATASETS/amalgam/Training and Validation/nofire"
    ]

    for folder in base_folders:
        resize_images_in_folder(folder, size=(512, 512))


Resized ../DATASETS/amalgam/Testing/fire/36171982712_d1839b35aa_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/abc180.jpg
Resized ../DATASETS/amalgam/Testing/fire/52230132421_efbcf13fe2_o.png
Resized ../DATASETS/amalgam/Testing/fire/36204002431_91b77a7dc5_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/50517815722_17ae2240d2_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/9598063779_5a497ff60e_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/41094811384_1382be1721_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/28431967367_37bc44b0c1_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/51284080730_6ab71a3862_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/35531872313_eb8772df72_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/50575966873_8cef16dcc6_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/37342469502_36f0ec280c_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/abc181.jpg
Resized ../DATASETS/amalgam/Testing/fire/28020330864_d7a2429a7e_o.jpg
Resized ../DATASETS/amalgam/Testing/fire/37342470222_39ab

Step 2: Use pytorch to train model for recognizing fire/nofire image

In [15]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import os
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_data = ImageFolder("../DATASETS/amalgam/Testing", transform=transform)
val_data = ImageFolder("../DATASETS/amalgam/Training and Validation", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = total_loss / total
    train_accuracy = correct / total

    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_accuracy = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f} - Train acc: {train_accuracy:.4f} - Val loss: {val_loss:.4f} - Val acc: {val_accuracy:.4f}")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
acc = accuracy_score(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)
print(f"Test Accuracy: {acc:.4f}")


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

Save model!

In [13]:
# Save model
torch.save(model.state_dict(), "resnet18_amalgam.pth")


In [ ]:
import torch
from torchvision import models, transforms
import torch.nn as nn
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predict_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("resnet18_amalgam.pth", map_location=device))
model = model.to(device)
model.eval()

class_names = ['fire', 'nofire']
def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    image = predict_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        pred = torch.argmax(output, dim=1).item()
    return class_names[pred]

result = predict("../DATASETS/amalgam/Testing/fire/9570772331_0822c9b670_o.jpg")
print("Prediction:", result)


Prediction: fire


In [23]:
from PIL import Image

def predict(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        pred = torch.argmax(output, dim=1).item()
        return train_data.classes[pred]

predict("../DATASETS_ORIG/forest_fire/Training and Validation/nofire/abc197.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dristiroy/DATASCIFORESTFIRES/DataSciForestFires/DATASETS_ORIG/forest_fire/Training and Validation/nofire/abc197.jpg'